# 10 Aves Dataset

In this case we use more classes

In [1]:
import pandas as pd
import numpy as np

import torchaudio
import torchaudio.transforms as T

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

import librosa

In [2]:
df = pd.read_csv("../datasets/AnimalSoundFull.csv")
df.head()

,gbifID,identifier,species,genus,family,class,phylum,file_name
0,1572324720,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...
1,1572324719,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...
2,1572324718,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...
3,1572324717,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...
4,1572324716,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...


In [13]:
families = ["Alaudidae", "Bucerotidae", "Cardinalidae", "Cisticolidae", "Corvidae",
           "Estrildidae", "Falconidae", "Laridae", "Phasianidae", "Thraupidae"]

In [15]:
df_10aves = df[df["family"].isin(families)].reset_index(drop=True)
df_10aves.head()

,gbifID,identifier,species,genus,family,class,phylum,file_name
0,1291058075,http://www.tierstimmenarchiv.de/recordings/Cot...,Coturnix coturnix,Coturnix,Phasianidae,Aves,Chordata,Chordata/Aves/Phasianidae/Coturnix/Coturnix_co...
1,1269848202,http://www.tierstimmenarchiv.de/recordings/Tet...,Tetrastes bonasia,Tetrastes,Phasianidae,Aves,Chordata,Chordata/Aves/Phasianidae/Tetrastes/Tetrastes_...
2,1269848190,http://www.tierstimmenarchiv.de/recordings/Ste...,Sterna paradisaea,Sterna,Laridae,Aves,Chordata,Chordata/Aves/Laridae/Sterna/Sterna_paradisaea...
3,1269848189,http://www.tierstimmenarchiv.de/recordings/Ste...,Sterna paradisaea,Sterna,Laridae,Aves,Chordata,Chordata/Aves/Laridae/Sterna/Sterna_paradisaea...
4,1269848188,http://www.tierstimmenarchiv.de/recordings/Ste...,Sterna paradisaea,Sterna,Laridae,Aves,Chordata,Chordata/Aves/Laridae/Sterna/Sterna_paradisaea...


In [16]:
df_10aves.shape

(1267, 8)

In [17]:
num_samples = df_10aves.shape[0]
np.random.seed(42)


df_10aves = df_10aves.drop(columns=["identifier",
                                    "species",
                                    "genus",
                                    "class",
                                    "phylum"
                                   ])
df_10aves

,gbifID,family,file_name
0,1291058075,Phasianidae,Chordata/Aves/Phasianidae/Coturnix/Coturnix_co...
1,1269848202,Phasianidae,Chordata/Aves/Phasianidae/Tetrastes/Tetrastes_...
2,1269848190,Laridae,Chordata/Aves/Laridae/Sterna/Sterna_paradisaea...
3,1269848189,Laridae,Chordata/Aves/Laridae/Sterna/Sterna_paradisaea...
4,1269848188,Laridae,Chordata/Aves/Laridae/Sterna/Sterna_paradisaea...
...,...,...,...
1262,779844894,Alaudidae,Chordata/Aves/Alaudidae/Alauda/Alauda_arvensis...
1263,779844893,Alaudidae,Chordata/Aves/Alaudidae/Alauda/Alauda_arvensis...
1264,779844270,Bucerotidae,Chordata/Aves/Bucerotidae/Aceros/Aceros_corrug...
1265,779844269,Bucerotidae,Chordata/Aves/Bucerotidae/Aceros/Aceros_corrug...


In [24]:
def getSpectrogram(row):
  wf, sample_rate = torchaudio.load("../data/" + row.file_name)
  
  n_fft = 1024
  win_length = None
  hop_length = 512
  n_mels = 128

  mel_spectrogram = T.MelSpectrogram(
      sample_rate=sample_rate,
      n_fft=n_fft,
      win_length=win_length,
      hop_length=hop_length,
      center=True,
      pad_mode="reflect",
      power=2.0,
      norm='slaney',
      onesided=True,
      n_mels=n_mels,
      mel_scale="htk",
  )
  
  melspec = mel_spectrogram(wf)[0]
  
  height = 128*2
  width = height*4
  dpi = 100
  
  fig = plt.figure(frameon=False, figsize=(width/dpi, height/dpi), dpi=dpi)

  
  ax = plt.Axes(fig, [0., 0., 1., 1.])
  ax.set_axis_off()
  fig.add_axes(ax)
  
  im = ax.imshow(librosa.power_to_db(melspec), origin='lower', aspect="auto")
  
  file_name = "../spectrograms/10aves_dataset/" + str(row["gbifID"]) + ".jpg"
  
  plt.savefig(file_name)
  plt.close()
      
  return

In [25]:
tqdm.pandas(desc="Creating Spectrograms")

In [26]:
_ = df_10aves.progress_apply(getSpectrogram, axis=1)

Creating Spectrograms:   0%|          | 0/1267 [00:00<?, ?it/s]

/home/pheithar/.local/lib/python3.9/site-packages/torchaudio/functional/functional.py:432: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (513) may be set too low.
  warnings.warn(


In [28]:
df_10aves.to_csv("../datasets/10Aves.csv", index=False)